# Tokenize and save data

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.text import *

In [3]:
from fastai.text import data

In [4]:
import pandas as pd
import numpy as np
from fastai_data import *

## Preparing the data

In [5]:
version = 'v9'
data_path = Path('data/midi')
version_path = data_path/version

In [6]:
source_dir = 'midi_encode/np/shortdur'
out_path = version_path/source_dir
csv_path = out_path/f'midi_encode.csv'
version_path.ls()

[PosixPath('data/midi/v9/metadata'),
 PosixPath('data/midi/v9/midi_sources'),
 PosixPath('data/midi/v9/midi_encode'),
 PosixPath('data/midi/v9/midi_npz'),
 PosixPath('data/midi/v9/midi_transform')]

In [7]:
csv = pd.read_csv(csv_path)
csv = csv.loc[csv[source_dir].notna()];

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,4,15,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
def create_databunch(files, cache_name, batch_size=32, load_cached=False):
    if load_cached and (out_path/f'{cache_name}/train_ids.npy').exists():
        data = LMNPDataBunch.load(out_path, bs=batch_size, cache_name=cache_name)
    else:
        ps = [OpenNPFileProcessor()]

        data = (ItemList(items=files, path=out_path, processor=ps)
                .split_by_rand_pct(0.01, seed=6)
                .label_const(label_cls=LMLabelList))
        data.x._bunch = LMNPDataBunch
        data = data.databunch(bs=batch_size)
        data.save(cache_name)
    return data

In [9]:
def get_files(csv):
    files = csv[source_dir]
    flist = [Path(version_path/f) for f in files.values]
    flist = [f for f in flist if f.exists()]
    return flist

In [10]:
version_path

PosixPath('data/midi/v9')

### Create All Dataset

In [11]:
csv

,inferred_offset,song_url,instruments,ht_mode,midi_title,title,seconds,midi,inferred_key,quarter_length,...,ht_time_signature,bpm,section,parts,genres,mxl,midi_transform,midi_npz,midi_npz_timesteps,midi_encode/np/shortdur
1,0.0,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",1.0,yu-gi-oh3,yu-gi-oh-theme-song,15.000000,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,32.0,...,4.0,128.00,chorus,"intro,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,129.0,midi_encode/np/shortdur/hooktheory/pianoroll/w...
2,5.0,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",1.0,kiefer,kiefer,10.000000,midi_sources/hooktheory/pianoroll/w/what-a-day...,E minor,16.0,...,4.0,96.00,chorus,chorus,Jazz,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...,65.0,midi_encode/np/shortdur/hooktheory/pianoroll/w...
4,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Weezer - Fall Together,fall-together-,10.322581,midi_sources/hooktheory/pianoroll/w/weezer/fal...,A minor,16.0,...,4.0,93.00,chorus,chorus,Rock,NaN,midi_transform/hooktheory/pianoroll/w/weezer/f...,midi_npz/hooktheory/pianoroll/w/weezer/fall-to...,65.0,midi_encode/np/shortdur/hooktheory/pianoroll/w...
5,-5.0,https://www.hooktheory.com/theorytab/view/wavv...,"Piano,Piano",1.0,afraidofheightsintro,afraid-of-heights,8.135593,midi_sources/hooktheory/pianoroll/w/wavves/afr...,F major,16.0,...,4.0,118.00,intro,"intro,verse,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wavves/a...,midi_npz/hooktheory/pianoroll/w/wavves/afraid-...,65.0,midi_encode/np/shortdur/hooktheory/pianoroll/w...
6,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Buddy Holly,buddy-holly,16.000000,midi_sources/hooktheory/pianoroll/w/weezer/bud...,C major,32.0,...,4.0,120.00,chorus,"verse,pre-chorus,chorus,bridge,solo","Pop,Rock",NaN,midi_transform/hooktheory/pianoroll/w/weezer/b...,midi_npz/hooktheory/pianoroll/w/weezer/buddy-h...,129.0,midi_encode/np/shortdur/hooktheory/pianoroll/w...
7,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,greatest man,the-greatest-man-that-ever-lived,24.615385,midi_sources/hooktheory/pianoroll/w/weezer/the...,C major,32.0,...,4.0,78.00,solo-1,"intro,solo-1",NaN,NaN,midi_transform/hooktheory/pianoroll/w/weezer/t...,midi_npz/hooktheory/pianoroll/w/weezer/the-gre...,129.0,midi_encode/np/shortdur/hooktheory/pianoroll/w...
8,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,falling for you intro,falling-for-you,11.111111,midi_sources/hooktheory/pianoroll/w/weezer/fal...,C major,20.0,...,4.0,108.00,intro,"intro,verse,chorus,solo",NaN,NaN,midi_transform/hooktheory/pianoroll/w/weezer/f...,midi_npz/hooktheory/pianoroll/w/weezer/falling...,81.0,midi_encode/np/shortdur/hooktheory/pianoroll/w...
9,0.0,https://www.hooktheory.com/theorytab/view/wavv...,"Piano,Piano",1.0,afraidverse,afraid-of-heights,17.454545,midi_sources/hooktheory/pianoroll/w/wavves/afr...,C major,32.0,...,4.0,110.00,verse,"intro,verse,chorus",NaN,NaN,midi_transform/hooktheory/pianoroll/w/wavves/a...,midi_npz/hooktheory/pianoroll/w/wavves/afraid-...,129.0,midi_encode/np/shortdur/hooktheory/pianoroll/w...
10,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Buddy Holly,buddy-holly,7.868852,midi_sources/hooktheory/pianoroll/w/weezer/bud...,A minor,16.0,...,4.0,122.00,verse,"verse,pre-chorus,chorus,bridge,solo","Pop,Rock",NaN,midi_transform/hooktheory/pianoroll/w/weezer/b...,midi_npz/hooktheory/pianoroll/w/weezer/buddy-h...,65.0,midi_encode/np/shortdur/hooktheory/pianoroll/w...
11,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,intro greatest man,the-greatest-man-that-ever-lived,25.600000,midi_sources/hooktheory/pianoroll/w/weezer/the...,C major,32.0,...,4.0,75.00,intro,"intro,solo-1",NaN,NaN,midi_transform/hooktheory/pianoroll/w/weezer/t...,midi_npz/hook

In [12]:
all_files = get_files(csv)
all_data = create_databunch(all_files, cache_name='tmp/all', load_cached=True)

/home/ubuntu/fastai/fastai/data_block.py:202: UserWarning: `random_split_by_pct` is deprecated, please use `split_by_rand_pct`.
  warn("`random_split_by_pct` is deprecated, please use `split_by_rand_pct`.")


In [13]:
import random
sample_data = create_databunch(random.sample(all_files, 1000), cache_name='tmp/sample')

Note: we are reusing all_vocab for the following datasets

### Create Hooktheory Dataset

In [14]:
hook_csv = csv.loc[csv.source.isin(['hooktheory', 'cprato'])]
hook_files = get_files(hook_csv)

In [15]:
hook_data = create_databunch(hook_files, cache_name='tmp/hook')

### Create Pop Dataset

In [16]:
pop_csv = csv.loc[csv.source.isin(['midiworld', 'freemidi', 'wikifonia', 'lmd_clean'])]
pop_files = get_files(pop_csv)

#### Save vocab sizes

In [13]:
max_vocab_file = out_path/'tmp/all/max_vocab.npy'
if max_vocab_file.exists():
    VOCAB_SZ = np.load(max_vocab_file).tolist()
else:
    train_ids_file = out_path/'tmp/all/train_ids.npy'
    all_ids = np.load(train_ids_file)
    id_cat = np.concatenate(all_ids); id_cat.shape
    ax = tuple(range(len(id_cat.shape)-1))
    max_vocab = id_cat.max(axis=ax)+1
    if max_vocab[0] in [118, 120]: max_vocab[0] += 12 # to allow data augmentation
    np.save(max_vocab_file, max_vocab)
    VOCAB_SZ = max_vocab.tolist()
VOCAB_SZ

[118, 132]

#### Create sample

In [18]:
pop_data = create_databunch(pop_files, cache_name='tmp/pop')

### Create Classical Dataset

In [19]:
clc_csv = csv.loc[csv.source.isin(['classical_piano', 'ecomp'])]
clc_files = get_files(clc_csv)

In [20]:
clc_data = create_databunch(clc_files, cache_name='tmp/clc')

In [21]:
len(clc_files)

2848

## Create dumpster dataset

In [22]:
dmp_csv = csv.loc[csv.source.isin(['reddit'])]
dmp_files = get_files(dmp_csv)

In [23]:
dmp_data = create_databunch(dmp_files, cache_name='tmp/dmp')

### Testing

In [24]:
data = pop_data

In [25]:
data.train_ds.x

ItemList (15588 items)
[[ 2  0]
 [ 1  3]
 [64  7]
 [ 1  6]
 ...
 [75  5]
 [ 1  4]
 [73 27]
 [64 27]],[[ 2  0]
 [ 1  3]
 [71  6]
 [71  6]
 ...
 [ 1  3]
 [59  4]
 [ 1  3]
 [59  4]],[[ 2  0]
 [ 1  3]
 [46  7]
 [ 1 10]
 ...
 [ 1  4]
 [64  5]
 [59  5]
 [52  5]],[[ 2  0]
 [ 1  3]
 [52  7]
 [43  7]
 ...
 [48  7]
 [44  7]
 [43  7]
 [43  7]],[[ 2  0]
 [ 1  3]
 [63 15]
 [51 15]
 ...
 [ 1  4]
 [89  7]
 [79  7]
 [46  7]]
Path: data/midi/v9/midi_encode/np/shortdur

In [12]:
train_ids_file = out_path/'tmp/all/train_ids.npy'
all_ids = np.load(train_ids_file)
id_cat = np.concatenate(all_ids); id_cat.shape
ax = tuple(range(len(id_cat.shape)-1))
max_vocab = id_cat.max(axis=ax)
max_vocab = (max_vocab+1).tolist(); max_vocab

[118, 132]

In [27]:
from encode_data import *

In [29]:
one_text = data.train_ds[0][0].text

AttributeError: 'numpy.ndarray' object has no attribute 'text'

In [ ]:
seq = str2seq(one_text); seq

In [ ]:
seq2numpy(seq).shape

In [ ]:
s = str2stream(one_text)

In [ ]:
s.show('midi')

In [ ]:
s.show('text')